### Loading Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

### Loading Dataset




In [3]:
df = pd.read_csv("/content/drive/MyDrive/NLP/train.csv") 
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
#Removing null values
df = df.dropna()
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
df['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [8]:
# get the independent features
X = df.drop(['label'],axis=1)

#get the dependent features
y = df['label']

In [9]:
tf.__version__

'2.8.2'

In [10]:
# vocabulary size
voc_size=5000

In [11]:
message = X.copy()
message.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [12]:
message.reset_index(inplace=True)

In [13]:
#Downloading stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
ps = PorterStemmer()
lm = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

corpus = []
for i in range(0, len(message)):
  review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
  review = review.lower()
  review = review.split()

  # review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = [lm.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [15]:
corpus[0]

'house dem aide even see comey letter jason chaffetz tweeted'

### One Hot





In [16]:
onehot_rep = [one_hot(word,voc_size) for word in corpus]
onehot_rep[0]

[4734, 34, 3482, 4790, 1225, 3618, 25, 1218, 2687, 3127]

### Padding

In [17]:
sent_length= 20
embedded_docs = pad_sequences(onehot_rep,padding='post',maxlen=sent_length)
embedded_docs[0]

array([4734,   34, 3482, 4790, 1225, 3618,   25, 1218, 2687, 3127,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

### Creating Model

In [18]:
embedding_vector_features=100
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 100)           500000    
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 580,501
Trainable params: 580,501
Non-trainable params: 0
_________________________________________________________________
None


### Train Test Split




In [19]:
X_train, X_test, y_train, y_test = train_test_split(embedded_docs,y, test_size=0.33, random_state=42)

### Model Training

In [20]:
#Final model traainning
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 9ms/step - loss: 0.3160 - accuracy: 0.8540 - val_loss: 0.2001 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1352 - accuracy: 0.9498 - val_loss: 0.2241 - val_accuracy: 0.9114
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0838 - accuracy: 0.9708 - val_loss: 0.2408 - val_accuracy: 0.9130
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0554 - accuracy: 0.9820 - val_loss: 0.3177 - val_accuracy: 0.9130
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0358 - accuracy: 0.9888 - val_loss: 0.3662 - val_accuracy: 0.9082
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0265 - accuracy: 0.9917 - val_loss: 0.4101 - val_accuracy: 0.9089
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0195 - accuracy: 0.9934 - val_loss: 0.4293 - val_accuracy: 0.9001
Epoch 

### Accuracy Check

In [21]:
y_pred=model.predict(X_test)

y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [22]:
accuracy_score(y_test,y_pred)

0.8974316487158244

In [23]:
confusion_matrix(y_test,y_pred)

array([[3152,  267],
       [ 352, 2264]])